In [29]:
import pandas as pd
from pymongo import MongoClient
import plotly.express as px

def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]

db = _connect_mongo(host='localhost', port=27017, username=None, password=None, db='OpenRoadDB')

KPI = db.kpi
ENGINE = db.engine_history

# run_id = ['IibZFXkk0xZQ'] # greedy
# run_id = ['y0iaD5HLrUX5'] # random
# run_id = ['JmHIEWPtC2Oe'] # Compromise

run_id_meta = {
    'LJUtgcGOnlMn': 'Greedy',
#     'y0iaD5HLrUX5': 'Random',
    'sISFMWyhDPmk': 'Compromise',
}




In [30]:
def get_cumulative_pivot(collection, run_id_meta, metric):
    cursor = collection.find({
            'run_id': {'$in': [k for k, _ in run_id_meta.items()]},
            'metric': metric
        },
        projection={ '_id': 0, 'run_id': 1, 'sim_clock': 1, 'value': 1,},
        sort=[('sim_clock', 1)]
    )

    metric_df = pd.DataFrame(list(cursor))

    # revenue_df['cum_revenue'] = revenue_df['value'].cumsum()
    # px.line(revenue_df, x='sim_clock', y='cum_revenue', color=run_id)

    metric_pivot = pd.pivot_table(metric_df, index='sim_clock', columns='run_id', values='value')
    metric_pivot = metric_pivot.cumsum().rename(columns=run_id_meta)

    return metric_pivot




In [31]:
sum_metric = ['served', 
              'revenue', 
              'cancelled', 
             ]
avg_metric = ['wait_time_driver_confirm', 
              'wait_time_assignment',
             ]

for m in sum_metric:
    metric_pivot = get_cumulative_pivot(KPI, run_id_meta, m)

    fig = px.line(metric_pivot, title=m) #, labels = run_id_meta)
    fig.show()


served_pivot = get_cumulative_pivot(KPI, run_id_meta, 'served')
for m in avg_metric:
    metric_pivot = get_cumulative_pivot(KPI, run_id_meta, m) / served_pivot

    fig = px.line(metric_pivot, title=m) #, labels = run_id_meta)
    fig.show()


    

In [39]:

def get_engine_perf(collection, run_id_list):
    cursor = collection.find({
            'run_id': {'$in': run_id_list},
        },
        projection={ '_id': 0, 'online_params': 1, 'sim_clock': 1, 'runtime_performance': 1,},
        sort=[('sim_clock', 1)]
    )

    metric_df = pd.DataFrame(list(cursor))
    metric_df = pd.concat([metric_df.drop(['online_params'], axis=1), metric_df['online_params'].apply(pd.Series)], axis=1)
    metric_df = pd.concat([metric_df.drop(['runtime_performance'], axis=1), metric_df['runtime_performance'].apply(pd.Series)], axis=1)

    metric_df

    return metric_df



engine_df = get_engine_perf(ENGINE, [k for k, v in run_id_meta.items() if v == 'Compromise'])
# engine_df.tail(100)
fig = px.line(engine_df, x='sim_clock', 
              y=['realtimePickupTime','realtimeRevenue', 'realtimeServiceScore'],
              title='Compromise') #, labels = run_id_meta)
fig.show()
